In [1]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import time
import wave
import IPython.display as ipd
import librosa.display
import sklearn
from scipy.stats import kurtosis, skew
import csv
import os
import pandas as pd

class Audio():
    def __init__(self):
        super(Audio, self).__init__()
        
        self.SHORT_NORMALIZE = (1.0/32768.0)
        self.RATE = 44100  
        self.INPUT_BLOCK_TIME = 0.05
        self.CHUNK = int(self.RATE*self.INPUT_BLOCK_TIME)
        self.CHANNEL = 2
        
        self.microphone_name = "3- Andrea PureAudio"

    def initiate_mic(self):
        self.pa = pyaudio.PyAudio()
        device_index = self.discover_mic()  
        self.mic = self.pa.open(format = pyaudio.paInt16,
                    channels = self.CHANNEL,
                    rate =  self.RATE ,
                    input = True,
                    input_device_index = device_index,
                    frames_per_buffer = self.CHUNK
                    )
        
    def discover_mic(self):

        for i in range(self.pa.get_device_count() ):     
            devinfo = self.pa.get_device_info_by_index(i)   

            if self.microphone_name in devinfo["name"]:
                    device_index = i
                    return device_index

    def record_audio(self, filename, runtime):

        blocks = []

        print("Recording starts..")
        st = time.time()
        while True:
            block = self.mic.read(self.CHUNK)
            blocks.append(block)

            t = time.time() - st

            if t>runtime:        
                self.save_audio(filename, blocks)        
                self.close_mic()
                break

        print("Recording done! Saved as ", filename)

    def save_audio(self, filename, blocks):
        wf = wave.open(filename, 'wb')
        wf.setnchannels(self.CHANNEL)
        wf.setsampwidth(self.pa.get_sample_size(pyaudio.paInt16))
        wf.setframerate(self.RATE)
        wf.writeframes(b''.join(blocks))
        wf.close()

    def close_mic(self):
        self.mic.stop_stream()
        self.mic.close()
        self.pa.terminate()  
        
class Features():
    def __init__(self):
        super(Features, self).__init__()
        
    def load_audio_file(self, filename):
        self.x, self.sr = librosa.load(filename)
        return self.x, self.sr
    
    def load_data_file(self, path):
    
        if os.path.exists(path):
            return pd.read_csv(path, index_col=False)

        else:
            header = 'filename spec_centroid spec_rolloff spec_bandwidth rmse chroma tempogram max_amplitude min_amplitude \
             mean_amplitude standard_deviation variance skew kurtosis zero_crossing_rate quantiles_10 quantiles_25 \
             quantiles_50 quantiles_75'

            for i in range(1, 20):
                header += f' mfcc{i}'
            header += ' coordinate_x'
            header += ' coordinate_y'
            header += ' coordinate_z'
            header += ' direction'
            header = header.split()

            file = open(path, 'w', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(header)

            return pd.read_csv(path, index_col=False)

    def tempogram(self):
        oenv = librosa.onset.onset_strength(y=self.x, sr=self.sr)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=self.sr)       
        return tempogram

    def get_audio_features(self):
        feature = {}
        
        feature['spec_centroid'] = np.mean(librosa.feature.spectral_centroid(y=self.x, sr=self.sr))
        feature['spec_rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=self.x, sr=self.sr))
        feature['spec_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=self.x, sr=self.sr))
        feature['rmse'] = np.mean(librosa.feature.rms(y=self.x))
        feature['chroma'] = np.mean(librosa.feature.chroma_stft(y=self.x, sr=self.sr))
        feature['tempogram'] = np.mean(self.tempogram())
        feature['max_amplitude'] = np.mean(np.max(self.x))
        feature['min_amplitude']  = np.mean(np.min(self.x))
        feature['mean_amplitude']  = np.mean(np.mean(self.x))
        feature['standard_deviation']  =  np.std(self.x)
        feature['variance']  = np.var(self.x)
        feature['skew']  = skew(self.x)
        feature['kurtosis']  = kurtosis(self.x)
        feature['zero_crossing_rate']  = sum(librosa.zero_crossings(self.x, pad=False))
        feature['quantiles_10']  = np.quantile(self.x, 0.10)
        feature['quantiles_25']  = np.quantile(self.x, 0.25)
        feature['quantiles_50']  = np.quantile(self.x, 0.50)
        feature['quantiles_75']  = np.quantile(self.x, 0.75)
        
        mfcc = librosa.feature.mfcc(y=self.x, sr=sr)
        for i in range(1, 20):
            feature[f'mfcc{i}'] = np.mean(mfcc[i])
        
        return feature
    
    def get_location_coordinate(self):
        coords = {}
        coords['coordinate_x'] = 0
        coords['coordinate_y'] = 0
        coords['coordinate_z'] = 0
        
        return coords
    
    def get_direction(self):
        direction = {'direction': 'NE'}
        return direction

## Record audio

In [ ]:
filename = "test.wav"
runtime = 5 #sec

audio = Audio()
audio.initiate_mic()
audio.record_audio(filename, runtime)
ipd.Audio(filename)

## Create features from audio and add it to file

In [3]:
f = Features()

path = 'data/audio_feature.csv'
df = f.load_data_file(path)

files = ["test.wav", "test.wav","test.wav"]
for file in files:
    x, sr = f.load_audio_file(file)
    audio_feat = f.get_audio_features()
    audio_feat_df = pd.DataFrame([list(audio_feat.values())], columns = list(audio_feat.keys()))
    audio_feat_df.insert(loc=0, column='filename', value=[file])
    
    coordinates = f.get_location_coordinate()
    coord_df = pd.DataFrame([list(coordinates.values())], columns = list(coordinates.keys()))
    
    direction = f.get_direction()
    direction_df =  pd.DataFrame([list(direction.values())], columns = list(direction.keys()))
    
    row = pd.concat([audio_feat_df, coord_df, direction_df], axis=1)
    
    df = pd.concat([df, row], axis=0)
    df.to_csv(path, index=False)

## Model

In [4]:
f = Features()
path = 'data/audio_feature.csv'
df = f.load_data_file(path)

In [5]:
df

,filename,spec_centroid,spec_rolloff,spec_bandwidth,rmse,chroma,tempogram,max_amplitude,min_amplitude,mean_amplitude,...,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,coordinate_x,coordinate_y,coordinate_z,direction
0,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
1,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
2,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
3,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
4,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
5,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
6,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
7,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE
8,test.wav,906.152371,1821.748861,1776.2221,0.036051,0.651856,0.087105,1.012027,-1.016979,0.000937,...,5.27814,5.890125,5.496608,5.802282,6.628213,5.592276,0,0,0,NE


In [11]:
model_input = df.iloc[:, 1:-4]
coord_x = df['coordinate_x']
direction = df['direction']

target = direction

In [12]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(model_input, target, test_size=0.3, random_state=1) # 70% training and 30% test

# Create classifer object
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
knn = KNeighborsClassifier(n_neighbors=6)

clfs = [knn, rf, dt]

for clf in clfs:
    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    
    print(y_pred)

#     matrix = confusion_matrix(y_test, y_pred)
#     print(matrix.diagonal()/matrix.sum(axis=1))

['NE' 'NE' 'NE']
['NE' 'NE' 'NE']
['NE' 'NE' 'NE']
